### 거래주체(기관, 외인)과 주가의 관계
- 가설1 : 외인, 기관의 매수, 매도량이 급증했을 때 주가가 상승 or 하락?

In [2]:
import pandas as pd
import numpy as np
import time
import pickle
from datetime import datetime
from tqdm import tqdm
#from scipy.stats.mstats import gmean

In [3]:
import pymysql
from sqlalchemy import create_engine
from pandas.io import sql

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly.graph_objs as go
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [5]:
PASSWORD=""

### Import data

In [6]:
conStock = pymysql.connect(
    user='root', 
    passwd=PASSWORD, 
    host='127.0.0.1', 
    db='stock', 
    charset='utf8'
)

In [7]:
conPrice = pymysql.connect(
    user='root', 
    passwd=PASSWORD, 
    host='127.0.0.1', 
    db='stock_price', 
    charset='utf8'
)

In [8]:
def allCodes():
    '''
    데이터베이스에 저장된 코드(799개) return
    '''
    cur = conStock.cursor()
    cur.execute('show tables')
    codes = [c[0] for c in cur]
    return codes

In [9]:
def extractDf(cur, code):
    '''
    코드를 입력받아 해당하는 dataframe 반환
    '''
    sql = f"select * from {code}"
    cur.execute(sql)
    result = cur.fetchall();

    df = pd.DataFrame(result)
    columnNames = [x[0] for x in cur.description]
    df.columns = columnNames
    return df

### Preprocessing

In [10]:
class Preprocessor:
    '''
    데이터의 scaling과 관련된 메소드
    '''
    def normalize(arr):
        #     sign = np.sign(arr)
        #     arr = np.log(np.abs(arr)+1)*sign
        return (arr-np.mean(arr))/np.std(arr)
    
    def minmax(arr):
        return((arr-np.min(arr))/(np.max(arr)-np.min(arr)))

In [11]:
class TradeStradegy:
    '''
    mergePrice: 기관, 외인 데이터베이스로부터 추출된 데이터프레임과
    주가 데이터베이스로부터 추출된 데이터프레임 결합
    
    stradegy1Condition: 기관, 외인 순매매량의 이상치 index 반환
    
    stradegy1: 위의 index를 기준으로 n일 후 종가 변화량들 반환
    '''
    def __init__(self, code, criteria, window, dayLater, who):
        self.code = code
        self.criteria = criteria
        self.window = window
        self.dayLater = dayLater
        self.who = who
        self.curC = conStock.cursor()
        self.curP = conPrice.cursor()
        self.df = self.mergePrice()
        
    def mergePrice(self):
        dfWho = extractDf(self.curC, self.code)
        dfPrice = extractDf(self.curP, self.code)
        
        dfWho['Date'] = pd.to_datetime(dfWho['Date'])
        dfWho['volumePrice'] = dfWho[self.who]*dfWho['Close']
        dfWho = dfWho.drop(columns = ['Close','Change','Volume'])
        df = dfWho.merge(dfPrice,how='left',on='Date')
        df = df[::-1].reset_index(drop=True)
        return(df)
    
    
    def stradegy1Condition(self):
        ppsr = Preprocessor
        self.df['volumePriceRolling'] = ppsr.normalize(self.df['volumePrice'].rolling(window=self.window).mean())
        if(self.criteria>0):
            plusIdx = self.df[self.df['volumePriceRolling']>self.criteria].index.values
        else:
            plusIdx = self.df[self.df['volumePriceRolling']<self.criteria].index.values
        
        if(len(plusIdx)==0):
            return np.array([])
        uniqueIdx = [plusIdx[0]]
        for i in range(1,len(plusIdx)):
            if(plusIdx[i] - plusIdx[i-1] > 5):
                uniqueIdx.append(plusIdx[i])
        return np.array(uniqueIdx)
    
    def stradegy1(self): #or inst_sum
        uniqueIdx = self.stradegy1Condition()
        uniqueIdxLater = uniqueIdx + self.dayLater

        uniqueIdx = uniqueIdx[uniqueIdxLater<len(self.df)]
        uniqueIdxLater = uniqueIdxLater[uniqueIdxLater<len(self.df)]
        
        
        dfOrigin = self.df.iloc[uniqueIdx,:]
        dfLater = self.df.iloc[uniqueIdxLater,:]
        return (dfLater['Close'].values/dfOrigin['Close'].values,
                np.random.choice(self.df[self.dayLater:]['Close'].values / self.df[:-1*self.dayLater]['Close'].values, len(uniqueIdx)))
#        return (1-np.mean(dfSelected['Close']/dfSelected['Open']), len(dfSelected))

### 간단한 EDA

In [12]:
codes = allCodes()

In [13]:
tss = TradeStradegy('c005930') #대한항공의 주식코드
df = tss.mergePrice()
dfPlot = df[['Date','Close','inst_sum','foreign_sum']]

In [14]:
dfPlot.iplot(x='Date', y=['inst_sum','foreign_sum'], title = "기관, 외국인의 일일 순매매량")

In [15]:
dfPlot1 = dfPlot[::-1]
dfPlot1.iloc[:,2:] = dfPlot1.iloc[:,2:].cumsum()

#scaling for plotting
for i in range(1,dfPlot1.shape[1]):
        dfPlot1.iloc[:,i] = Preprocessor.minmax(dfPlot1.iloc[:,i])

In [16]:
dfPlot1.iplot(x='Date', y=['inst_sum','foreign_sum','Close'], title = "기관, 외국인 누적 순매매량(Scaled) + 종가(삼성전자)")

### 통계적 검정

### Strategy1
외국인 or 기관의 매도량이 급증 or 급락했을때 n일 후 주가의 변화


In [65]:
tss = TradeStradegy('c005930', 2.5, 10, 30, "inst_sum")
df = tss.mergePrice()

fig = df.iplot(x='Date', y='volumePrice', asFigure=True, title='삼성전자 기관 순매매량')
fig.show()

In [48]:
plusIdx = tss.stradegy1Condition()
dfSelected = tss.stradegy1()
np.mean(dfSelected)

In [74]:
ppsr = Preprocessor
df['vPR'] = df['volumePrice'].rolling(window=10).mean()
df['vPRNM'] = ppsr.normalize(df['vPR'])

fig = df['vPRNM'].iplot(asFigure=True, title='삼성전자 기관 순매매량(10일 이동평균 + 정규화)')
for i in plusIdx:
   # fig.add_scatter(x=[i]*100, y=np.linspace(-100000,100000,100), opacity=0.3)
    fig.add_trace(go.Scatter(
    x=[i]*100, y=np.linspace(df['vPRNM'].min(),df['vPRNM'].max(),100),
    opacity=0.3,
    marker_color='rgba(0, 0, 158, .8)'
))
fig.show()

In [68]:
df['vPRMM'] = ppsr.minmax(df['vPRNM'])
df['CloseMM'] = ppsr.minmax(df['Close'])

In [80]:
fig = df.iplot(asFigure=True, y='CloseMM', title = "삼성전자 기관 순매매량 급증시기와 종가")
for i in plusIdx:
   # fig.add_scatter(x=[i]*100, y=np.linspace(-100000,100000,100), opacity=0.3)
    fig.add_trace(go.Scatter(
    x=[i]*100, y=np.linspace(df['vPRMM'].min(),df['vPRMM'].max(),100),
    opacity=0.3,
    marker_color='rgba(0, 0, 158, .8)'
))
fig.show()

In [82]:
result = np.array([])
resultRandom = np.array([])
for code in tqdm(codes):
    tss = TradeStradegy(code, 2.5, 10, 30, "inst_sum")
    stdg1 = tss.stradegy1()
    changeSelected = stdg1[0]
    changeRandom = stdg1[1]
    result = np.append(result, changeSelected)
    resultRandom = np.append(resultRandom, changeRandom)

100%|██████████| 799/799 [01:24<00:00,  9.47it/s]


In [91]:
len(result)
np.mean(result)

3521

In [ ]:
n = 30
allCases = np.array([])
for code in tqdm(codes):
    df = extractDf(conPrice.cursor(), codes[0])
    allCases = np.append(allCases, df[n:]['Close'].values/df[:-n]['Close'].values)
    

In [96]:
# df = pd.DataFrame({"mean": meanList, "gmean": gmeanList})

# df.iplot(kind='histogram', theme='white', title='temp')

In [97]:
allCases_ = allCases - 1
result_ = result - 1

In [98]:
n = len(result)
pd.DataFrame({"random": np.random.choice(allCases_, n), "selected": result_}).iplot(kind='histogram', theme='white', title='Strategy1 vs random')

In [119]:
# bootstrap

meanList = []
#gmeanList = []
for i in range(10000):
    temp = np.random.choice(allCases, n)
    meanList.append(np.mean(temp))
#    gmeanList.append(gmean(temp))

In [122]:
min(meanList)

1.0126067157474217

## Parameter tuning

In [20]:
# 40이 끝
criteriaList = [-2.5, 2.5]
windowList = [10, 30]
dayLaterList = [1, 5, 10, 30, 60]
whoList = ['inst_sum', 'foreign_sum']
resultList = []
i=0
for cri in criteriaList:
    for win in windowList:
        for day in dayLaterList:
            for who in whoList:
                resultDic = {}
                resultDic['criteria'] = cri
                resultDic['window'] = win
                resultDic['dayLater'] = day
                resultDic['who'] = who
                print(i, end="-")
                i+=1
                result = np.array([])
                for code in codes:
                    tss = TradeStradegy(code, cri, win, day, who)
                    stdg1 = tss.stradegy1()
                    changeSelected = stdg1[0]
                    result = np.append(result, changeSelected)
                resultDic['result'] = result
                resultList.append(resultDic)
                
                

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-

In [24]:
with open("resultList.pkl", "wb") as f:
    pickle.dump(resultList, f)

In [111]:
with open("resultList.pkl", "rb") as f:
    resultList = pickle.load(f)

In [112]:
temp = resultList.copy()

In [114]:
for i in range(len(resultList)):
    temp[i]['result'] = gmean(temp[i]['result'])

In [116]:
len(codes)

799

In [115]:
pd.DataFrame(temp).sort_values("result")

,criteria,window,dayLater,who,result
19,-2.5,30,60,foreign_sum,0.928171
39,2.5,30,60,foreign_sum,0.928737
29,2.5,10,60,foreign_sum,0.932580
9,-2.5,10,60,foreign_sum,0.934047
38,2.5,30,60,inst_sum,0.938219
28,2.5,10,60,inst_sum,0.950252
37,2.5,30,30,foreign_sum,0.958950
27,2.5,10,30,foreign_sum,0.959273
7,-2.5,10,30,foreign_sum,0.959383
8,-2.5,10,60,inst_sum,0.959970


In [22]:
resultList[0]

{'criteria': -2.5,
 'window': 10,
 'dayLater': 1,
 'who': 'inst_sum',
 'result': array([0.98969072, 1.01470588, 0.94537815, ..., 0.99053628, 1.01595745,
        0.99048626])}

In [88]:
for i in range(len(resultList)):
    resultList[i]['result'] = np.mean(resultList[i]['result'])

In [89]:
pd.DataFrame(resultList)

,criteria,window,dayLater,who,result
0,-2.5,10,1,inst_sum,0.994343
1,-2.5,10,1,foreign_sum,1.004012
2,-2.5,10,5,inst_sum,0.992890
3,-2.5,10,5,foreign_sum,1.013319
4,-2.5,30,1,inst_sum,0.999794
5,-2.5,30,1,foreign_sum,1.019713
6,-2.5,30,5,inst_sum,1.004899
7,-2.5,30,5,foreign_sum,1.022198
8,2.5,10,1,inst_sum,1.000926
9,2.5,10,1,foreign_sum,1.008536


In [ ]:
#trash
class TradeStradegy:
    '''
    mergePrice : 기관, 외인 데이터베이스로부터 추출된 데이터프레임과
    주가 데이터베이스로부터 추출된 데이터프레임 결합
    
    stradegy1 : 기관, 외인 순매매 데이터를 이용한 매매 전략(다음 포스트에서 다룰 예정)
    '''
    def __init__(self, code):
        self.code = code
        self.curC = conStock.cursor()
        self.curP = conPrice.cursor()
        self.df = self.mergePrice()
        
    def mergePrice(self):
        dfWho = extractDf(self.curC, self.code)
        dfPrice = extractDf(self.curP, self.code)
        
        dfWho['Date'] = pd.to_datetime(dfWho['Date'])
        dfWho = dfWho.drop(columns = ['Close','Change','Volume'])
        df = dfWho.merge(dfPrice,how='left',on='Date')
        return(df)
    
    def stradegy1(self, criteria, dayLater, who="foreign_sum"): #or inst_sum
        ppsr = Preprocessor
        if(criteria>0):
            plusIdx = self.df[ppsr.normalize(self.df[who]*self.df['Close'])>criteria].index.values - dayLater
        else:
            plusIdx = self.df[ppsr.normalize(self.df[who]*self.df['Close'])<criteria].index.values - dayLater
        plusIdx=plusIdx[plusIdx>0]
        
        dfSelected =self.df.iloc[plusIdx,:]
        return (1-np.mean(dfSelected['Close']/dfSelected['Open']), len(dfSelected))